<a href="https://colab.research.google.com/github/beifa/M5_Forecasting/blob/master/eval_lags_m5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gc
import numpy as np
import pandas as pd

PATH =  '/kaggle/input/m5-forecasting-accuracy/'
PATH_AGG = '/kaggle/input/eval-default-data-m5/'

In [ ]:
#default reduce mem
def reduce_mem_usage(df, verbose=True):
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2    
  for col in df.columns: 
    col_type = df[col].dtypes
    if col_type in numerics: 
      c_min = df[col].min()
      c_max = df[col].max()
      if str(col_type)[:3] == 'int':
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
          df[col] = df[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
          df[col] = df[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
          df[col] = df[col].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
          df[col] = df[col].astype(np.int64)  
      else:
        if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
          df[col] = df[col].astype(np.float16)
        elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
          df[col] = df[col].astype(np.float32)
        else:
          df[col] = df[col].astype(np.float64)    
  end_mem = df.memory_usage().sum() / 1024**2
  if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
  return df

In [ ]:
import time
df = pd.read_pickle(PATH_AGG + 'default_m5.pkl')
df = df[['id', 'd', 'sales']]

# Lags
# with 28 day shift
start_time = time.time()
print('Create lags')

LAG_DAYS = [col for col in range(28,28+15)]
df = df.assign(**{
        '{}_lag_{}'.format(col, l): df.groupby(['id'])[col].transform(lambda x: x.shift(l))
        for l in LAG_DAYS
        for col in ['sales']
    })

# Minify lag columns
for col in list(df):
    if 'lag' in col:
        df[col] = df[col].astype(np.float16)

print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

# Rollings
# with 28 day shift
start_time = time.time()
print('Create rolling aggs')

for i in [7,14,30,60,180]:
    print('Rolling period:', i)
    df['rolling_mean_'+str(i)] = df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(i).mean()).astype(np.float16)
    df['rolling_std_'+str(i)]  = df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(i).std()).astype(np.float16)

# Rollings
# with sliding shift
for d_shift in [1,7,14]: 
    print('Shifting period:', d_shift)
    for d_window in [7,14,30,60]:
        col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
        df[col_name] = df.groupby(['id'])['sales'].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)
    
    
print('%0.2f min: Lags' % ((time.time() - start_time) / 60))
df = reduce_mem_usage(df, verbose=True)
df.iloc[:, 3:].to_pickle('lags_m5.pkl')
print('Saved')

Create lags
7.35 min: Lags
Create rolling aggs
Rolling period: 7
Rolling period: 14
Rolling period: 30
Rolling period: 60
Rolling period: 180
Shifting period: 1
Shifting period: 7
Shifting period: 14
17.53 min: Lags
Mem. usage decreased to 3643.40 Mb (0.0% reduction)
Saved
